# MERGE013
Best of Each Type as of July 14

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import os
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [2]:
tracking = pd.read_csv('../tracking/tracking.csv')

In [3]:
tracking.head()

,run_id,model_number,eval_metric,n_estimators,early_stopping_rounds,random_state,n_threads,learning_rate,n_fold,model_type,...,2JHNcv_f3,2JHC_features,2JHC_meta_fc_mae_cv_f3,2JHC_f3_best_iter,2JHCcv_f3,3JHH_features,3JHH_meta_fc_mae_cv_f3,3JHH_f3_best_iter,3JHHcv_f3,3JHC_features
0,0610_1341,M010,mae,500000.0,500.0,529.0,32.0,0.5,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0611_0308,M012,mae,500000.0,500.0,529.0,32.0,0.5,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0611_1250,M013,mae,50000.0,500.0,529.0,32.0,0.5,5.0,catboost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0611_1249,M014,mae,500000.0,500.0,529.0,24.0,0.3,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0613_0402,M015,mae,500000.0,500.0,529.0,24.0,0.3,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for cv in ['1JHC_mae_cv', '2JHH_mae_cv', '1JHN_mae_cv',
          '2JHN_mae_cv', '2JHC_mae_cv', '3JHH_mae_cv',
          '3JHC_mae_cv', '3JHN_mae_cv']:
    print(f'----------{cv}----------')
    print(tracking[['model_number','run_id','model_type', cv]] \
          .set_index('model_number') \
          .sort_values(cv) \
          .head())

----------1JHC_mae_cv----------
                 run_id model_type  1JHC_mae_cv
model_number                                   
M047          0714_2226   catboost       0.4504
M046_1JHC     0712_2026   catboost       0.4976
M043          0710_2124   catboost       0.5124
M045_1JHC     0711_2313   catboost       0.5244
M044          0711_1936   catboost       0.5376
----------2JHH_mae_cv----------
                 run_id model_type  2JHH_mae_cv
model_number                                   
M047          0714_2226   catboost       0.1098
M046          0714_0836   catboost       0.1251
M043          0710_2124   catboost       0.1382
M038          0707_0907   catboost       0.1405
M040          0708_1639   catboost       0.1421
----------1JHN_mae_cv----------
                 run_id model_type  1JHN_mae_cv
model_number                                   
M047          0714_2226   catboost       0.3276
M046_1JHN     0713_2329   catboost       0.3566
M030          0701_1300       lgbm      

In [5]:
types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC','3JHH','3JHC', '3JHN']
oof = pd.DataFrame()
sub = pd.DataFrame()
fi = pd.DataFrame()
for t in types:
    for file in os.listdir(f'../type_results/{t}/'):
        if 'M047' in file:
            if 'oof' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                oof = pd.concat([oof, df])
            if 'sub' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                sub = pd.concat([sub, df])
            if 'fi' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                fi = pd.concat([fi, df])

In [6]:
types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC','3JHH','3JHC', '3JHN']
oof46 = pd.DataFrame()
sub46 = pd.DataFrame()
fi46 = pd.DataFrame()
for t in types:
    for file in os.listdir(f'../type_results/{t}/'):
        if 'M046' in file:
            if 'oof' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                oof46 = pd.concat([oof46, df])
            if 'sub' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                sub46 = pd.concat([sub46, df])
            if 'fi' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                fi46 = pd.concat([fi46, df])

In [16]:
oof = pd.concat([oof, oof46.loc[~oof46['type'].isin(oof['type'].unique())]])
sub = pd.concat([sub, sub46.loc[~sub46['type'].isin(sub['type'].unique())]])
fi = pd.concat([fi, fi46.loc[~fi46['type'].isin(fi['type'].unique())]])

In [17]:
for i, d in oof.groupby('type'):
    print(i)
    print(mean_absolute_error(d['scalar_coupling_constant'], d['oof_preds']))
    
glmae = group_mean_log_mae(oof['scalar_coupling_constant'], oof['oof_preds'], oof['type'])
print('\nGroup LMAE')
print(glmae)

1JHC
0.4503699367015481
1JHN
0.3275648329377529
2JHC
0.17372169526194634
2JHH
0.10975865602221349
2JHN
0.11998399872086996
3JHC
0.2216202227144778
3JHH
0.12251207685277472
3JHN
0.10535325576960344

Group LMAE
-1.7313370208850738


# Compare with Good Sub

In [18]:
sub_good = pd.read_csv('../submissions/BLEND008_submission_-1.7087CV_Jul14best.csv')

In [19]:
sub['good_scc'] = sub_good['scalar_coupling_constant']

In [20]:
sub[['scalar_coupling_constant','good_scc']].corr()

,scalar_coupling_constant,good_scc
scalar_coupling_constant,1.000000,0.999987
good_scc,0.999987,1.000000


In [21]:
sub = sub.sort_values('id').reset_index(drop=True)
oof = oof.sort_values('id').reset_index(drop=True)

In [22]:
# Save Results
sub[['id','scalar_coupling_constant']].to_csv('../submissions/MERGE014_sub_M046_M047_-1.7313CV.csv', index=False)
oof.to_csv('../oof/MERGE014_oof_M046_M047_-1.7313CV.csv', index=False)